# Assignment 1
## Shuai Shao

In [1]:
import numpy as np
import imageio.v3 as iio
from feat import Detector
from IPython.display import Image
import csv

ModuleNotFoundError: No module named 'numpy'

In [6]:

with open("../dataset/annotations.csv", mode='r') as annotations:
    reader = csv.reader(annotations, delimiter=',')
    for files in reader:
        print(files)



#detector = Detector(face_model="retinaface", au_model = )

['file', 'valence']
['thumbs-up', 'positive']
['sad-man', 'negative']
['claws', 'negative']
['laughing-couple', 'positive']
['by-the-sea', 'negative']
['arguing', 'negative']
['enjoying-the-sun', 'positive']
['piggyback', 'positive']
['back-off', 'negative']
['handshake', 'positive']
['tablet', 'negative']
['happy-man', 'positive']
['sad-woman', 'negative']
['business', 'positive']
['pain', 'negative']
['students', 'positive']
['bills', 'negative']
['bullied', 'negative']
['disagreement', 'negative']
['grandparents', 'positive']


## Reflections
### 1.


### 2.
* I would use backward feature elimination to determine which features yields the best performance increase / lowest performance decrease at the specified number of features. While it may be tempting to choose the AUs with the greatest mean difference between classes, these AUs may be intercorrelated, meaning that using more than one would may provide little to new information gain.
* Having too many features can be a problem as the solution space becomes much larger and requires more data in order to train the model, otherwise possibly leading to overfitting, known as the curse of dimensionality.